# Práctica 1: Criptografía clásica
## UAM, 2022/2023

### Maitane Gómez González
### Ana Martínez Sabiote

## 1. Sustitución monoalfabeto

## 1.a Método afín
El siguiente programa implementa el método afín.

Llamada a la función:

afin {-C|-D} {-m |Zm|} {-a N×} {-b N+} [-i f ilein] [-o f ileout]


In [18]:
import gmpy2
from gmpy2 import mpz

In [19]:
"""
VERSIÓN SIN GMP 

def algoritmo_euclides(a,b):
    if a%b == 0:
        return b
    else:
        return algoritmo_euclides(b, a%b)
"""

'\nVERSIÓN SIN GMP \n\ndef algoritmo_euclides(a,b):\n    if a%b == 0:\n        return b\n    else:\n        return algoritmo_euclides(b, a%b)\n'

In [20]:
def algoritmo_euclides(a,b):
    if gmpy2.t_mod(a,b) == 0:
        return b
    else:
        return algoritmo_euclides(b, gmpy2.t_mod(a,b))

In [21]:
mcd=algoritmo_euclides(39,150)
print(mcd)

3


In [22]:
algoritmo_euclides(7,15)

mpz(1)

In [23]:
"""
VERSIÓN SIN GMP 

def algoritmo_euclides_extendido(a,b):

    # Identidad de Bézout 1=u*a + v*b
    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v
    if a==0:
        mcd=b
        u=0
        v=1
    else:
        mcd, x, y = algoritmo_euclides_extendido(b%a, a)
        u=y-(b//a)*x
        v=x
        
    return mcd, u, v
"""

'\nVERSIÓN SIN GMP \n\ndef algoritmo_euclides_extendido(a,b):\n\n    # Identidad de Bézout 1=u*a + v*b\n    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v\n    if a==0:\n        mcd=b\n        u=0\n        v=1\n    else:\n        mcd, x, y = algoritmo_euclides_extendido(b%a, a)\n        u=y-(b//a)*x\n        v=x\n        \n    return mcd, u, v\n'

In [24]:
def algoritmo_euclides_extendido(a,b):
    """
    # Condición a>b, sino las cambiamos
    if b>a:
        aux=a
        a=b
        b=aux
    """
    # Identidad de Bézout 1=u*a + v*b
    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v
    if a==0:
        mcd=b
        u=0
        v=1
    else:
        mcd, x, y = algoritmo_euclides_extendido(gmpy2.c_mod(b,a), a)
        u=gmpy2.sub(y,(gmpy2.mul(gmpy2.c_div(b,a),x)))
        v=x
        
    return mcd, u, v

In [25]:
def inverso(a,m):
    result = algoritmo_euclides_extendido(a,m)
    # Comprobamos que el mcd es 1 para que exista inverso multiplicativo
    # En consecuencia, a y m determinan una función afín inyectiva
    if result[0] == 1:
        # Entonces devolvemos el coeficiente u (que acompaña a) de la Id. de Bézout
        inv=result[1]
        return inv
    else:
        print("Error")

In [26]:
def read_input(i):
    # Primero tomamos el input de i o de la entrada estándar
    if i==0:
        cadena=input()
    else:
        file=open(i, "r")
        cadena=file.read()
        file.close()
    print("Cadena: {}".format(cadena))
    return cadena

In [27]:
def print_output(o,cadena):
    if o==0:
        print("Cadena: {}".format(cadena))
    else:
        file=open(o, "w")
        cadenaToStr = ' '.join([str(elem) for elem in cadena])
        file.write(cadenaToStr)
        file.close()

In [28]:
def afin(modo,m,a,b,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if algoritmo_euclides(a,m) == 1:
        if modo=="-C":
            cadena=read_input(i)
            #Traducimos los caracteres a números
            cadena_numerica=[]
            for k in cadena:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
            cadena_cifrada=[]
            for k in cadena_numerica:
                cadena_cifrada.append(((a*k)+b)%m)
            print_output(o,cadena_cifrada)
        elif modo=="-D":
            cadena_cifrada=read_input(i)
            cadena_cifrada=cadena_cifrada.split(", ")
            cadena_descifrada=[]
            cadena_texto=[]
            inv=inverso(a,m)
            for i in range(len(cadena_cifrada)):
                cadena_cifrada[i]=int(cadena_cifrada[i])
            for k in cadena_cifrada:
                k_descifrado=gmpy2.c_mod(gmpy2.mul((k-b),inv),m)
                if k_descifrado<0:
                    k_descifrado=m+k_descifrado
                cadena_descifrada.append(k_descifrado)
                cadena_texto.append(alfabeto[k_descifrado])
            
            print_output(o, cadena_texto)
    else:
        print("{} y {} no son primos relativos. Error".format(a,m))

In [29]:
afin("-C",52,23,3,"cadena.txt")

Cadena: Hola Nueva York!
Cadena: [34, 13, 48, 3, 16, 47, 43, 18, 3, 9, 13, 30, 25]


In [30]:
afin("-C",130,16,27)

16 y 130 no son primos relativos. Error


In [31]:
afin("-C",52,23,3)

Universidad Autonoma de Madrid
Cadena: Universidad Autonoma de Madrid
Cadena: [21, 42, 31, 18, 43, 30, 1, 31, 20, 3, 20, 29, 47, 24, 13, 42, 13, 19, 3, 20, 43, 45, 3, 20, 30, 31, 20]


In [32]:
afin("-D",52,23,3)

21, 42, 31, 18, 43, 30, 1, 31, 20, 3, 20, 29, 47, 24, 13, 42, 13, 19, 3, 20, 43, 45, 3, 20, 30, 31, 20
Cadena: 21, 42, 31, 18, 43, 30, 1, 31, 20, 3, 20, 29, 47, 24, 13, 42, 13, 19, 3, 20, 43, 45, 3, 20, 30, 31, 20
Cadena: ['U', 'n', 'i', 'v', 'e', 'r', 's', 'i', 'd', 'a', 'd', 'A', 'u', 't', 'o', 'n', 'o', 'm', 'a', 'd', 'e', 'M', 'a', 'd', 'r', 'i', 'd']


In [33]:
afin("-D",52,23,3,"cadena_cifrada.txt","resultado.txt")

Cadena: 34, 13, 48, 3, 16, 47, 43, 18, 3, 9, 13, 30, 25


## 1.b Criptoanálisis del cifrado afín

## 2. Sustitución polialfabeto

## 2.a Método de Hill

In [34]:
import numpy as np
import os
import math

In [35]:
def determinante(matriz):
   
    if len(matriz)==2 and len(matriz[0]):
        #calculamos el determinante
        det=matriz[0][0]*matriz[1][1]-matriz[1][0]*matriz[0][1]
       
        return det
    else:
        suma=0
        for i in range(len(matriz)): #calculamos el determinante por cofactores
            maux=copy.deepcopy(matriz)
            maux.remove(matriz[0]) #eliminamos la primera fila
            for j in range(len(maux)):
                maux[j]=maux[j][0:i]+maux[j][i+1:]
                
         
            suma= suma+ (-1)**((i+j)%2)*matriz[0][i]*determinante(maux)
            
        return suma
        

In [36]:
matriz = [[1,2,3], [3,5,5], [1,2,3]]
print(determinante(matriz))

NameError: name 'copy' is not defined

In [ ]:
def inversa(matriz,modulo):
    det=np.det(matriz)%modulo
    adjunto=np.adj(matriz)%modulo
    return=np.

In [ ]:
def adjunto(matriz):
    adjunto=np.zeros(np.shape(matriz))
    if len(matriz)==2 and len(matriz[0]):
         #calculamos el adjunto
        adjunto[0][1]=-matriz[1][0]
        adjunto[1][0]=-matriz[0][1]
        adjunto[0][0]=matriz[1][1]
        adjunto[1][1]=matriz[0][0]
        
        return adjunto
    else:
        for i in range(len(matriz)):
            maux=copy.deepcopy(matriz)
            for j in range(len(matriz)):
             
                maux=np.delete(matriz,i,0)
                aux=np.delete(maux,j,1)
                auxi=maux.tolist()
           
                adjunto[i][j]=(-1)**((i+j)%2)*determinante(auxi)
                
        return adjunto
        

In [ ]:
matriz = [[1,2,3], [3,5,5], [1,2,3]]
print(adjunto(matriz))

In [ ]:
arr = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
np.delete(arr, 1, 0)

In [ ]:
matriz = [[1, 2, 4], [3,5,4], [1]]

# Padding a la matriz
n=3
print(matriz)
print(len(matriz[n-1]))
k=len(matriz[n-1])
if k<n:
    padding=[0]*(n)
    for i in range(0,k):
        padding[i]=matriz[n-1][i]
    matriz[n-1]=padding
  
print(matriz)
matriz=np.array(matriz)
matriz%3

In [ ]:
print(matriz[1])

In [ ]:
def hill(modo,m,n,k,i,o):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    matriz = np.loadtxt(k, delimiter = '\ t')
    det = np.linalg.det(matriz)%m # Hay que sustituir con nuestro propio método
    dim = np.shape(matriz)
    
    # Padding a la matriz
    k=len(matriz[n-1])
    if k<n:
        padding=[0]*(n)
        for i in range(0,k):
            padding[i]=matriz[n-1][i]
        matriz[n-1]=padding
    matriz=np.array(matriz)
    
    if algoritmo_euclides(det,m)==1:
        if modo=="-C":
            cadena=read_input(i)
            #Traducimos los caracteres a números
            cadena_numerica=[]
            for k in cadena:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
            # Dividimos en bloques de n elementos el texto
            # Si m no es múltiplo de n se añade padding
            m=len(cadena_numerica)/n
            maxi=len(cadena_numerica)
            matriz_numerica=np.zeros((math.ceil(m),n))
            pos=0
            for i in range(math.ceil(m)):
                for j in range(n):
                    if pos<maxi:
                        matriz_numerica[i][j]=cadena_numerica[pos]
                        pos=pos+1
    else:
        print("{} y {} no son primos relativos. Error".format(det,m))

In [ ]:
cadena_numerica=[2,4,5,3,6]
n=2

m=len(cadena_numerica)/n
maxi=len(cadena_numerica)
matriz_numerica=np.zeros((math.ceil(m),n))
pos=0
for i in range(math.ceil(m)):
    print("itero i")
    for j in range(n):
        print("itero j")
        if pos<maxi:
            print("meto valor")
            matriz_numerica[i][j]=cadena_numerica[pos]
            pos=pos+1
        
        
print(matriz_numerica)   
print(m)
print(math.ceil(m))
print(matriz_numerica.shape)
texto_numerico=np.concatenate(matriz_numerica)
print(texto_numerico)

## 2.b Método de Vigenere

In [37]:
def vigenere(modo,k,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    base=len(alfabeto)
    n=len(k)
    #Traducimos la clave de caracteres a números
    k_numerica=[]
    for j in k:
        if j in alfabeto: 
            k_numerica.append(alfabeto.index(j))
    if modo=="-C":
        cadena=read_input(i)
        # Traducimos los caracteres a números
        cadena_numerica=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_numerica.append(alfabeto.index(k))
        # Dividimos en bloques de n elementos el input
        # Si m no es múltiplo de n se añade padding
        m=len(cadena_numerica)/n
        maxi=len(cadena_numerica)
        matriz_numerica=np.zeros((math.ceil(m),n))
        pos=0
        for i in range(math.ceil(m)):
            for j in range(n):
                if pos<maxi:
                    matriz_numerica[i][j]=cadena_numerica[pos]
                    pos=pos+1
        # Tenemos una matriz que tenemos que cifrar. 
        # Cada bloque es una fila de la matriz
        filas=matriz_numerica.shape[0]
        elementos=matriz_numerica.shape[1]
        #print("Matriz numerica")
        #print(matriz_numerica)
        matriz_cifrada=np.zeros((filas,elementos))
        #print("Matriz cifrada de ceros")
        #print(matriz_cifrada)
        if elementos==n:
            print("Bloques ok")
        for i in range(filas):
            for j in range(elementos):
                matriz_cifrada[i][j]=(matriz_numerica[i][j]+k_numerica[j])%base
        #print("Matriz cifrada")
        #print(matriz_cifrada)
        cadena_cifrada=np.concatenate(matriz_cifrada)
        #print(cadena_cifrada)
        print_output(o,cadena_cifrada)
    elif modo=="-D":
        cadena_cifrada=read_input(i)
        cadena_cifrada=cadena_cifrada.split(" ")
        #for i in range(len(cadena_cifrada)):
        #   cadena_cifrada[i]=int(cadena_cifrada[i])
        # Dividimos en bloques de n elementos el texto cifrado
        # Si m no es múltiplo de n se añade padding
        m=len(cadena_cifrada)/n
        maxi=len(cadena_cifrada)
        matriz_cifrada=np.zeros((math.ceil(m),n))
        pos=0
        for i in range(math.ceil(m)):
            for j in range(n):
                if pos<maxi:
                    matriz_cifrada[i][j]=cadena_cifrada[pos]
                    pos=pos+1
        # Tenemos una matriz que tenemos que descifrar. 
        # Cada bloque es una fila de la matriz
        filas=matriz_cifrada.shape[0]
        elementos=matriz_cifrada.shape[1]
        matriz_descifrada=np.zeros((filas,elementos))
        if elementos==n:
            print("Bloques ok")
        for i in range(filas):
            for j in range(elementos):
                matriz_descifrada[i][j]=(matriz_cifrada[i][j]-k_numerica[j])%base
        cadena_descifrada=np.concatenate(matriz_descifrada)
        cadena_texto=[]
        for i in range(len(cadena_descifrada)):
            #cadena_cifrada[i]=int(cadena_cifrada[i])
            cadena_texto.append(alfabeto[int(cadena_descifrada[i])])
        print_output(o,cadena_texto)

In [39]:
vigenere("-C", "clave")

Universidad Autonoma de Madrid
Cadena: Universidad Autonoma de Madrid
Bloques ok
Cadena: [48. 24.  8. 42.  8. 19. 29.  8. 24.  4.  5. 37. 20. 40. 18. 15. 25. 12.
 21.  7.  6. 49.  0. 24. 21. 10. 14.  0. 21.  4.]


In [43]:
vigenere("-D", "clave")


21 10 14 0 21 4
Cadena: 21 10 14 0 21 4
Bloques ok
Cadena: ['t', 'Z', 'o', 'F', 'r', 'c', 'P', 'a', 'F', 'W']


In [41]:
vigenere("-C", "clave", "texto_vigenere.txt", "resultado_vigenere.txt")

Cadena: Universidad Autonoma de Madrid
Bloques ok


In [42]:
vigenere("-D", "clave", "resultado_vigenere.txt", "descifrado_vigenere.txt")

Cadena: 48.0 24.0 8.0 42.0 8.0 19.0 29.0 8.0 24.0 4.0 5.0 37.0 20.0 40.0 18.0 15.0 25.0 12.0 21.0 7.0 6.0 49.0 0.0 24.0 21.0 10.0 14.0 0.0 21.0 4.0
Bloques ok


## 2.c Criptoanálisis del cifrado de Vigenere

## 3. Cifrado de flujo

In [44]:
def rec_fib(n):
    if n > 1:
        return rec_fib(n-1) + rec_fib(n-2)
    return n

In [84]:
# Generador de secuencia aleatoria
def generador_aleatorio(m,cont):
    k=(rec_fib(m)%m)*m*cont
    return k

In [46]:
# Ejemplo de secuencia cifrante de 5 elementos para clave 14
m=14
for i in range(5):
    k=generador_aleatorio(m,i)
    print(k)

0
5278
10556
15834
21112


In [90]:
# m es la clave
# n es el tamaño de la secuencia de claves
def flujo(modo,m,n,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if modo=="-C":
        cadena=read_input(i)
        # Traducimos los caracteres a números
        cadena_numerica=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_numerica.append(alfabeto.index(k))
        # Ciframos carácter a carácter
        cadena_cifrada=[]
        count=0
        for i in cadena_numerica:
            if count<n:
                k=generador_aleatorio(m,count)
            else:
                k=generador_aleatorio(m,count-n)
            cadena_cifrada.append(int(bin(i^k)[2:]))
            count=count+1
        print_output(o,cadena_cifrada)
    elif modo=="-D":
        cadena_cifrada=read_input(i)
        cadena_cifrada=cadena_cifrada.split(" ")
        cadena_descifrada=[]
        cadena_texto=[]
        for i in range(len(cadena_cifrada)):
            cadena_cifrada[i]=int(cadena_cifrada[i],2)
        count=0
        for i in cadena_cifrada:
            if count<n:
                k=generador_aleatorio(m,count)
            else:
                k=generador_aleatorio(m,count-n)
            cadena_descifrada.append((i^k))
            count=count+1
        for i in range(len(cadena_descifrada)):
            cadena_texto.append(alfabeto[int(cadena_descifrada[i])])
        print_output(o,cadena_texto)

In [94]:
flujo("-C", 4,2)

Hola
Cadena: Hola
Cadena: [100001, 10, 1011, 1100]


In [95]:
flujo("-D", 4,2)

100001 10 1011 1100
Cadena: 100001 10 1011 1100
Cadena: ['H', 'o', 'l', 'a']


In [91]:
flujo("-C", 10,20)

Madrid
Cadena: Madrid
Cadena: [100110, 110010, 1100111, 10000111, 11000000, 11111001]


In [92]:
flujo("-D", 10, 20)

100110 110010 1100111 10000111 11000000 11111001
Cadena: 100110 110010 1100111 10000111 11000000 11111001
Cadena: ['M', 'a', 'd', 'r', 'i', 'd']
